In [46]:
import pandas as pd
import numpy as np
import pymysql
from sklearn.cluster import AgglomerativeClustering as AC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from matplotlib import pyplot as plt

In [5]:
connection = pymysql.connect(host="us-cdbr-iron-east-03.cleardb.net",
              user="bb8507107bd736",
              password="89bc7c14",
              db="heroku_94b4589a90ac9bc")

df = pd.read_sql("SELECT * FROM final_pitching_table", connection)
df.head()

,IP,SO,ER,W,IP_Avg,SO_Avg,ER_Avg,W_Avg,BatAvg1,BatAvg2,...,BatAvg9,FieldAvg1,FieldAvg2,FieldAvg3,FieldAvg4,FieldAvg5,FieldAvg6,FieldAvg7,FieldAvg8,FieldAvg9
0,6.0,4.0,1.0,0.0,5.05,4.250000,6.242500,0.250000,0.279591,0.320348,...,0.159143,0.989956,1.000000,0.984636,0.989542,1.000000,0.954143,0.991176,0.944316,1.000000
1,6.0,4.0,2.0,0.0,5.70,5.333333,1.046667,0.666667,0.719200,0.404316,...,0.186857,1.000000,1.000000,0.994522,0.846087,0.975533,1.000000,0.997333,1.000000,1.000000
2,5.0,5.0,3.0,0.0,5.86,4.600000,3.674000,0.200000,0.719200,0.202087,...,0.315400,1.000000,1.000000,0.994522,0.846087,0.994783,1.000000,0.997333,0.992312,1.000000
3,6.0,4.0,4.0,1.0,6.22,3.600000,4.652000,0.600000,0.242346,0.279393,...,0.244731,0.971111,0.998185,0.994333,1.000000,1.000000,0.928964,1.000000,0.940227,0.976903
4,4.0,7.0,5.0,0.0,6.02,4.200000,3.168000,0.400000,0.248889,0.285929,...,0.247348,0.995217,1.000000,0.981500,1.000000,1.000000,0.989280,0.995000,1.000000,1.000000


Get correlation of the dependent variables against every other variable.

In [6]:
df.corr()[['IP','SO','ER','W']]

,IP,SO,ER,W
IP,1.000000,0.423307,-0.439820,0.416834
SO,0.423307,1.000000,-0.245760,0.257580
ER,-0.439820,-0.245760,1.000000,-0.415735
W,0.416834,0.257580,-0.415735,1.000000
IP_Avg,0.254796,0.155030,0.028856,0.067351
SO_Avg,0.159163,0.362263,-0.102517,0.094667
ER_Avg,-0.057133,-0.094146,-0.011792,0.053238
W_Avg,0.035137,0.062354,0.071024,-0.044197
BatAvg1,0.004478,-0.035486,0.041874,-0.018587
BatAvg2,-0.018131,-0.046754,0.060384,-0.009154


In [74]:
scaler = MinMaxScaler()
batting_vars = df.filter(regex = 'Field')

batting_vars_scaled = batting_vars #pd.DataFrame(scaler.fit_transform(batting_vars), columns=batting_vars.columns)

N = 7

bat_var_cluster = AC(n_clusters=N).fit(batting_vars_scaled)

df['AC_Labels'] = pd.Series(bat_var_cluster.labels_ , index = df.index)

In [75]:
for i in range(N):
    
    print('cluster '+str(i)+': ', np.mean(df[ df['AC_Labels'] == i ]['ER']), len(df[df['AC_Labels'] == i ]))

cluster 0:  2.5717255717255716 481
cluster 1:  2.210526315789474 38
cluster 2:  1.8181818181818181 11
cluster 3:  2.5542168674698793 83
cluster 4:  0.6666666666666666 3
cluster 5:  2.488372093023256 172
cluster 6:  3.2142857142857144 14


In [63]:
df[ df['AC_Labels'] == 4]

,IP,SO,ER,W,IP_Avg,SO_Avg,ER_Avg,W_Avg,BatAvg1,BatAvg2,...,FieldAvg1,FieldAvg2,FieldAvg3,FieldAvg4,FieldAvg5,FieldAvg6,FieldAvg7,FieldAvg8,FieldAvg9,AC_Labels
22,7.0,6.0,1.0,1.0,NaN,NaN,NaN,NaN,0.307833,0.159000,...,0.993800,0.983842,0.993667,0.998100,0.965333,1.000000,0.976440,0.998000,0.976000,4
23,6.2,4.0,1.0,0.0,5.020,3.000000,4.570000,0.400000,0.307833,0.159000,...,0.993800,0.983842,0.993667,0.998100,0.965333,1.000000,0.976440,0.992800,0.976000,4
44,6.0,4.0,1.0,0.0,5.300,6.500000,3.350000,0.750000,0.307833,0.159000,...,1.000000,0.997231,0.978000,1.000000,1.000000,0.998000,0.995640,0.985286,0.941783,4
45,6.0,9.0,1.0,1.0,5.750,6.250000,4.720000,0.500000,0.159000,0.270286,...,0.978952,0.997231,0.978000,1.000000,1.000000,0.985286,1.000000,0.941783,0.998400,4
121,5.0,1.0,2.0,0.0,5.100,1.000000,3.380000,1.000000,0.225222,0.249187,...,1.000000,0.985962,0.931640,0.994652,0.971040,0.980263,0.967875,0.992389,0.667000,4
125,5.2,8.0,2.0,1.0,5.860,5.400000,2.974000,0.200000,0.225222,0.249187,...,1.000000,0.985962,0.931640,0.994652,0.971040,0.980263,0.967875,0.992389,1.000000,4
128,3.2,0.0,7.0,0.0,5.240,3.600000,4.012000,0.000000,0.225222,0.249187,...,1.000000,0.985962,0.931640,0.994652,0.971040,1.000000,0.967875,0.992389,1.000000,4
131,6.0,3.0,1.0,0.0,5.320,4.000000,3.580000,0.400000,0.225222,0.249187,...,1.000000,0.985962,0.931640,0.994652,0.971040,1.000000,0.967875,0.964833,1.000000,4
132,3.0,2.0,5.0,0.0,6.020,3.400000,3.970000,0.400000,0.249187,0.364125,...,0.995643,0.951389,0.997040,0.982083,1.000000,0.995577,0.997143,0.967700,1.000000,4
209,8.0,5.0,2.0,1.0,5.000,3.000000,5.480000,0.000000,0.259630,0.209833,...,0.975824,0.980105,1.000000,0.995913,0.983545,1.000000,0.837500,0.982720,1.000000,4
